# Check moms data against radial profiles

* load a moms data file
* radial plot of spherical average, compare with RProf 

In order to use the moms data with the ppm.py moms class the moms data written by the code need to be decompressed. See  `How-to-Decompress.md`  in `ppmstar/code/moments_data_reader_ppmsstar2.0` for instructions.

In order to know which of the 32 quantities in the code-written moms data are actually in the 10 decompressed variable slots you have to check the code in the `*DataAnalysis.F` file. The subroutine that actually does all of the work is `getmybqstuff` where the 10 `whatever` slots are set.

In [68]:
%pylab ipympl
import os,sys
from multiprocessing import Pool

ppmpy_dir = '/scratch/f/fherwig/fherwig/repos/PyPPM'
sys.path.insert(0,ppmpy_dir)
from ppmpy import ppm

# set cycling combination of color-blind labels, glyphs, styles
lll= 2*['-', '--', ':', '-.']
markers = ['X','h','<','>','s','^','d','X','p']
random.shuffle(lll)
CB_color_cycle = ['#4daf4a', '#a65628', '#984ea3',
                  '#ff7f00', '#f781bf', '#377eb8',
                  '#999999', '#e41a1c', '#dede00']
rc('axes', prop_cycle=(cycler('color', CB_color_cycle[0:8]) + cycler('marker',markers[0:8])+cycler('linestyle',lll)))
rc('axes', prop_cycle=(cycler('color', CB_color_cycle[0:8]) +cycler('linestyle',lll)))

# named tuple for using rprofs and momsdata
# moms data are almost always used alongside rprof data and therefore it is recommended 
# to create a convenient dictionary that will hold the rprof and moms instance 
import collections
hydro = collections.namedtuple('hydro', ['moms','rprof'])

# turn off matplotlib messages
logging.getLogger("matplotlib").setLevel(logging.CRITICAL)

Populating the interactive namespace from numpy and matplotlib


In [2]:
data_dir = '/scratch/f/fherwig/fherwig'
run_dir = 'M107'
moms_dir = os.path.join(data_dir,run_dir,'moms/myavsbq')
rprof_dir = os.path.join(data_dir,run_dir,'prfs')

At this point check what the decompressed variables are. For M107 these are
```Fortran
       thyng = max(fv(ibq,jbq,kbq),1.0e-12)

       whatever(ibq,jbq,kbq,1) = xc(ibq)
       whatever(ibq,jbq,kbq,2) = rhofux(ibq,jbq,kbq) * rhofinv
       whatever(ibq,jbq,kbq,3) = rhofuy(ibq,jbq,kbq) * rhofinv
       whatever(ibq,jbq,kbq,4) = rhofuz(ibq,jbq,kbq) * rhofinv
       whatever(ibq,jbq,kbq,5) = ut(ibq,jbq,kbq)
       whatever(ibq,jbq,kbq,6) = ur(ibq,jbq,kbq)
       whatever(ibq,jbq,kbq,7) = vort(ibq,jbq,kbq)
       whatever(ibq,jbq,kbq,8) = pf
       whatever(ibq,jbq,kbq,9) = rhof
       whatever(ibq,jbq,kbq,10) = thyng
```
which is the list below (except that we know that in the M100 series runs the vorticity is not calculated correctly, and can't be recovered, and therefore has to be computed from the velocity components).

In [3]:
# the list of variables that are stored within the momsdata cube.
var_list = ['xc','ux','uy','uz','|ut|','|ur|','|w|','P','rho','fv']

## Load data
### Load rprof and moms individually

In [36]:
# load moms instance
dump = 2000    # decompressed dumps in M107/moms/myavsbq start at dump 2000, check manually
moms = ppm.MomsDataSet(moms_dir,init_dump_read=dump,dumps_in_mem=2,var_list=var_list,rprofset=ppm.RprofSet(rprof_dir))

9711 rprof files found in '/scratch/f/fherwig/fherwig/M107/prfs/.
Dump numbers range from 201 to 9943.
7947 .aaa files found in '/scratch/f/fherwig/fherwig/M107/moms/myavsbq/.
Dump numbers range from 201 to 9943.


In [37]:
# load rprof data
rp_set = ppm.RprofSet(rprof_dir)
rp = rp_set.get_dump(dump)        # get one dump to access dump data, usually dump 0 should exist, but not for M107
Nx = rp.get('Nx')                 # get grid dimension
print(f'Nx = {Nx}')

9711 rprof files found in '/scratch/f/fherwig/fherwig/M107/prfs/.
Dump numbers range from 201 to 9943.
Nx = 768


### Load as dictionary

In [38]:
myrun = hydro(ppm.MomsDataSet(moms_dir,init_dump_read=dump,dumps_in_mem=2,var_list=var_list,rprofset=ppm.RprofSet(rprof_dir)),
              ppm.RprofSet(rprof_dir))

9711 rprof files found in '/scratch/f/fherwig/fherwig/M107/prfs/.
Dump numbers range from 201 to 9943.
7947 .aaa files found in '/scratch/f/fherwig/fherwig/M107/moms/myavsbq/.
Dump numbers range from 201 to 9943.
9711 rprof files found in '/scratch/f/fherwig/fherwig/M107/prfs/.
Dump numbers range from 201 to 9943.


## Basic grid properties

In [64]:
# 3D datacube of the cartesian coordinates at this dump
x,y,z = moms_grid = myrun.moms.get_cgrid()

# it is formatted such that var[z,y,x] refers to the variable at the (z,y,x) coordinates that those 
# indices correspond to. The indexing goes from negative to positive 
moms_ngridpoints = myrun.moms.moms_ngridpoints
print('The unique x,y,z coordinates are:')
print(x[np.random.randint(0,moms_ngridpoints), np.random.randint(0,moms_ngridpoints), :])

# to convert to spherical coordinates
r,theta,phi=myrun.moms.get_sgrid()

The unique x,y,z coordinates are:
[-2685.9375 -2657.8125 -2629.6875 -2601.5625 -2573.4375 -2545.3125
 -2517.1875 -2489.0625 -2460.9375 -2432.8125 -2404.6875 -2376.5625
 -2348.4375 -2320.3125 -2292.1875 -2264.0625 -2235.9375 -2207.8125
 -2179.6875 -2151.5625 -2123.4375 -2095.3125 -2067.1875 -2039.0625
 -2010.9375 -1982.8125 -1954.6875 -1926.5625 -1898.4375 -1870.3125
 -1842.1875 -1814.0625 -1785.9375 -1757.8125 -1729.6875 -1701.5625
 -1673.4375 -1645.3125 -1617.1875 -1589.0625 -1560.9375 -1532.8125
 -1504.6875 -1476.5625 -1448.4375 -1420.3125 -1392.1875 -1364.0625
 -1335.9375 -1307.8125 -1279.6875 -1251.5625 -1223.4375 -1195.3125
 -1167.1875 -1139.0625 -1110.9375 -1082.8125 -1054.6875 -1026.5625
  -998.4375  -970.3125  -942.1875  -914.0625  -885.9375  -857.8125
  -829.6875  -801.5625  -773.4375  -745.3125  -717.1875  -689.0625
  -660.9375  -632.8125  -604.6875  -576.5625  -548.4375  -520.3125
  -492.1875  -464.0625  -435.9375  -407.8125  -379.6875  -351.5625
  -323.4375  -295.3125  -267

## Get decompressed quantities and compare radial profiles with RProf
### Velocity and vorticity magnitude

In [63]:
dump=2000
# get the cartesian velocity components
ux = myrun.moms.get('ux',fname=dump)
uy = myrun.moms.get('uy',fname=dump)
uz = myrun.moms.get('uz',fname=dump)

# grab |U| on the entire grid, then get radial profile
U = myrun.moms.norm('ux','uy','uz')
U_moms, r_moms = myrun.moms.get_rprof(U,fname=dump)

# RProf data
r_rprof = myrun.rprof.get('R',fname=dump,resolution='l')
U_rprof = myrun.rprof.get('|U|',fname=dump,resolution='l')

# Plot
ifig=1;close(ifig);figure(ifig)
plot(r_moms, U_moms, label='MomsData')
plot(r_rprof, U_rprof, label='Rprof')
legend(); title('|U| velocity magnitude')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '|U| velocity magnitude')

In [77]:
# This cell could also be used to check radial and tangential velocity components

# vorticity from moms
w_moms, r_moms = myrun.moms.get_rprof('|w|',fname=dump)

# vorticity calculated from velocity components 
# from http://206-12-89-164.cloud.computecanada.ca/csa/hcorem25-paper1/-/blob/master/notebooks/calc_vortmags.py
def get_vel_rprofs_from_moms(star_moms, DUMP, things=['w', 'ur', 'ut'], rad_space=None, workers=1):
    '''                                                                                                 
    Calculates a quantities for Radial Profiles from the MomsDataSet instance                           
    for a given dump. Compatibility with multiprocessesing.                                             
                                                                                                        
    Parameters                                                                                          
    ----------                                                                                          
    star_moms : MomsDataSet                                                                             
        The MomsDataSet instance for which you want the radial profiles.                                
    DUMP : int                                                                                          
        The dump number for which to get radial profile                                                 
    things : list                                                                                       
        A subset of ['w', 'ur', 'ut'], for which to calculate the profiles.                             
    rad_space : float, None                                                                             
        The spacing between the radii for which to calcualte the spherical averages.                    
        If None, will take the moms grid size for the spacing.                                          
        The default is None.                                                                            
    workers : int, optional                                                                             
        Number of workers for multiprocessesing Pool.                                                   
        The default is 1. (Will not use multiprocessesing)                                              
                                                                                                        
    Returns                                                                                             
    -------                                                                                             
    numpy.ndarray                                                                                       
        2D array, first array is radii, the rest are pairs of computed and whatever.                    
    '''

    moms_grid = star_moms.get_cgrid()
    grid_size = abs(moms_grid[0][0][0][1] - moms_grid[0][0][0][0])
    moms_max_rad = max(moms_grid[0][0][0])

    xs = star_moms.get(0, fname=DUMP)
    ux = star_moms.get(1, fname=DUMP)
    uy = star_moms.get(2, fname=DUMP)
    uz = star_moms.get(3, fname=DUMP)

    ur, utheta, uphi = star_moms.get_spherical_components(ux, uy, uz)
    ut = np.sqrt(utheta**2 + uphi**2)

    moms_ut = star_moms.get(4, fname=DUMP)
    moms_ur = star_moms.get(5, fname=DUMP)
    moms_w  = star_moms.get(6, fname=DUMP)

    mX = moms_grid[0]
    mY = moms_grid[1]
    mZ = moms_grid[2]

    dux_dz, dux_dy, dux_dx = np.gradient(ux, moms_grid[0][0][0], moms_grid[0][0][0], moms_grid[0][0][0])
    duy_dz, duy_dy, duy_dx = np.gradient(uy, moms_grid[0][0][0], moms_grid[0][0][0], moms_grid[0][0][0])
    duz_dz, duz_dy, duz_dx = np.gradient(uz, moms_grid[0][0][0], moms_grid[0][0][0], moms_grid[0][0][0])

    curl = (duz_dy - duy_dz, dux_dz - duz_dx, duy_dx - dux_dy)
    vortmag = np.sqrt(curl[0]**2 + curl[1]**2 + curl[2]**2)

    if rad_space == None: rad_space = grid_size
    radii = np.arange(0, moms_max_rad, rad_space)

    global get_avg_velsvort
    def get_avg_velsvort(rad):
#         print("computing radius: {}".format(rad), end='\r')
        npoints = star_moms.sphericalHarmonics_lmax(rad)[-1]
        avg_w = 0; avg_moms_w = 0
        avg_ur = 0; avg_moms_ur = 0
        avg_ut = 0; avg_moms_ut = 0
        if 'ur' in things:
            sphere_ur = star_moms.get_spherical_interpolation(ur, rad, npoints=npoints)
            whatever_ur = star_moms.get_spherical_interpolation(moms_ur, rad, npoints=npoints)
            avg_ur = np.mean(sphere_ur)
            avg_moms_ur = np.mean(whatever_ur)
        if 'ut' in things:
            sphere_ut = star_moms.get_spherical_interpolation(ut, rad, npoints=npoints)
            whatever_ut = star_moms.get_spherical_interpolation(moms_ut, rad, npoints=npoints)
            avg_ut = np.mean(sphere_ut)
            avg_moms_ut = np.mean(whatever_ut)
        if 'w' in things:
            sphere_w = star_moms.get_spherical_interpolation(vortmag, rad, npoints=npoints)
            whatever_w = star_moms.get_spherical_interpolation(moms_w, rad, npoints=npoints)
            avg_w = np.mean(sphere_w)
            avg_moms_w = np.mean(whatever_w)
        return [rad, avg_w, avg_moms_w, avg_ur, avg_moms_ur, avg_ut, avg_moms_ut]

    if workers <= 1:
        return np.array(list(map(get_avg_velsvort, radii))).T

    pol = Pool(workers)
    results = pol.map(get_avg_velsvort, radii)
    return np.array(results).T

results = get_vel_rprofs_from_moms(myrun.moms, dump, things=['w'], rad_space=20, workers=39)
R_moms_calc,w_calc,w_moms_pool = results[0:3]



# Plot
ifig=1;close(ifig);figure(ifig)
plot(r_moms, w_moms, label='MomsData')
plot(R_moms_calc,w_calc, 'o',label='from vel components')
plot(R_moms_calc,w_moms_pool, 's',label='from moms averaging')
legend(); title('|U| velocity magnitude')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '|U| velocity magnitude')

For M107 this plot shows that the vorticity moms data (green dashed) is incorrect, an from these data sets the vorticiy must be calculated.

### Density, pressure and FV

The plots below show that in M107 the density and pressure are incorrect a well, but the fractional volume FV is correct.

In [56]:
# density radial profile
rho_moms, r_moms = myrun.moms.get_rprof('rho',fname=dump)
P_moms, r_moms = myrun.moms.get_rprof('P',fname=dump)

# FV radial profile
FV_moms, r_moms = myrun.moms.get_rprof('fv',fname=dump)

In [57]:
# get density, FV and R, all low resolution
r_rprof = myrun.rprof.get('R',fname=dump,resolution='l')
rho_rprof = (myrun.rprof.get('Rho0',fname=dump,resolution='l') + 
             myrun.rprof.get('Rho1',fname=dump,resolution='l'))
P_rprof = (myrun.rprof.get('P0',fname=dump,resolution='l') + 
             myrun.rprof.get('P1',fname=dump,resolution='l'))
FV_rprof = myrun.rprof.get('FV',fname=dump,resolution='l')

In [60]:
ifig=2;close(ifig);figure(ifig)
plot(r_moms,rho_moms,label='MomsData')
plot(r_rprof,rho_rprof,label='Rprof')
legend(); title('Density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Density')

In [61]:
ifig=3;close(ifig);figure(ifig)
plot(r_moms,P_moms,label='MomsData')
plot(r_rprof,P_rprof,label='Rprof')
legend(); title('Pressure')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Pressure')

In [62]:
ifig=4;close(ifig);figure(ifig)
plot(r_moms,FV_moms,label='MomsData')
plot(r_rprof,FV_rprof,label='Rprof')
legend(); title('FV')
yscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …